In [ ]:
%store -z

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno 
import seaborn as sns
import matplotlib.pyplot as plt 

#import warnings
#warnings.filterwarnings('ignore')



In [ ]:
# client data
df_clients =pd.read_csv('data/client_train.csv')

# invoice score
df_invoice =pd.read_csv('data/invoice_train.csv')

In [ ]:
df_clients.head()

In [ ]:
df_clients.info()

In [ ]:
df_invoice.head()

In [ ]:
df_invoice[['client_id', 'invoice_date', 'old_index', 'new_index']].query('client_id == "train_Client_0"').sort_values(by=['invoice_date'])

## Column descriptions:

### Client Data

* Client_id: Unique id for client
* District: District where the client is
* Client_catg: Category client belongs to
* Region: Area where the client is
* Creation_date: Date client joined
* Target: fraud:1 , not fraud: 0

### Invoice Data

* Client_id: Unique id for the client
* Invoice_date: Date of the invoice
* Tarif_type: Type of tax
* Counter_number:
* Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
* Counter_code:
* Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
* Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
* Consommation_level_1: Consumption_level_1
* Consommation_level_2: Consumption_level_2
* Consommation_level_3: Consumption_level_3
* Consommation_level_4: Consumption_level_4
* Old_index: Old index
* New_index: New index
* Months_number: Month number
* Counter_type: Type of counter

 the consumption level refers to the threshold of consumption to which a certain price is attributed

In [ ]:
#Y = coffee_quality["quality_score"]

# Data cleaningin and feature engineering

In [ ]:
df_clients.info()

In [ ]:
df_invoice.info()

In [ ]:
df_invoice.isnull().sum()

In [ ]:
df_invoice.describe().T

In [ ]:
df_joined =  pd.merge(df_clients, df_invoice, on="client_id", how="left")
#df_clients.join(df_invoice, on='client_id', how='left')

In [ ]:
df_joined.head(10)

In [ ]:
print(df_joined.client_id.nunique())
df_joined.counter_number.nunique()

In [ ]:
df_joined.nunique()

In [ ]:
df_joined.counter_type.unique()

In [ ]:
df_joined.client_catg.unique()

In [ ]:
df_joined.tarif_type.unique()

In [ ]:
df_joined.groupby('counter_statue').count()

In [ ]:
df_joined.groupby('counter_code').count()

In [ ]:
df_joined.counter_code.unique()

In [ ]:
df_joined.reading_remarque.unique()

In [ ]:
df_joined.groupby('reading_remarque').count()

In [ ]:
df_joined.counter_coefficient.unique()

In [ ]:
df_joined.old_index.unique()

In [ ]:
df_joined.info()

## To Dos

* remove client_id column
* turn into dummies: client_catg, tarif_type
* change target to 0,1 - DONE
* turn creation_date, invoice_date  into datetime or something else - DONE
* feature engineering: number of counters per client. then drop client_id, counter_number
* counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove - DONE
* counter_code: either dummies or find info on steg site or drop?
* reading_remarque: clean? turn into dummies
* counter_coefficient: try both?: treat numerically and as dummies?
* rescale consommation_level1 ... _4
* drop index old and new
* rescale months_number
* turn into dummy counter_type

In [ ]:
# check how many duplicated rows exist in the data frame
df_joined.duplicated().value_counts()

# there were 11 duplicatge rows, drop duplicates
df_joined.drop_duplicates(inplace=True)

In [ ]:
df_joined['counter_statue'].unique()

In [ ]:
# counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove

df_joined['counter_statue'] = df_joined['counter_statue'].map({
    '0': 0,
    0: 0, 
    1: 1,
    2 : 2,
    3: 3,
    4: 4,  
    5: 5,
    '5': 5,
    '1': 1,
    '4': 4, 
    'A': np.nan,
    618: np.nan, 
    269375: np.nan,
    46: np.nan, 
    420: np.nan,
    769: np.nan, 
    })
df_joined['counter_statue'].nunique()

In [ ]:
df_joined['counter_statue'].unique()

In [ ]:
# check for missing distribution

#msno.matrix(df_joined)

In [ ]:
# calculate fraction of data we would lose : 0.001%

print(f"numbers of rows : {df_joined.shape[0]}")
print(f"missing values in counter statue : {round(df_joined.counter_statue.isna().sum()/df_joined.shape[0]*100,4)} %")

# copy df
df_processed = df_joined.copy()
# drop NaN
df_processed.dropna(inplace=True, axis=0)


In [ ]:
df_joined.isna().sum()

In [ ]:
df_processed.isna().sum()

In [ ]:
df_processed['target'].unique()

In [ ]:
# change target from float to int (0,1)
df_processed.target = df_processed.target.astype(int)
df_processed.target.unique()

In [ ]:
df_processed.head()

In [ ]:
# turn columns invoice date and creation date to datetime

df_processed['invoice_date'] = pd.to_datetime(df_processed['invoice_date'], format='%Y-%m-%d')
df_processed['creation_date'] = pd.to_datetime(df_processed['creation_date'], format='%d/%m/%Y')

## Establishing Baseline Model

In [ ]:
df_processed.columns

In [ ]:
# heatmap
fig = plt.figure(figsize=(20,10))
sns.heatmap(df_processed[['disrict', 'client_catg', 'region', 'creation_date',
       'target', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number']].corr(), annot=True)



In [ ]:
# the pairplot took a lot of time so we did not see it so far and left it for now

# pairplot

# fig = plt.figure(figsize=(20,10))
# sns.pairplot(df_processed[['disrict', 'client_catg', 'region', 'creation_date', 'invoice_date', 'tarif_type', 'counter_number',
#        'counter_statue', 'counter_code', 'reading_remarque',
#        'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
#        'consommation_level_3', 'consommation_level_4', 'months_number', 'target']], hue='target')

In [ ]:

# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed, x = 'reading_remarque', hue='target', stat='percent');

In [ ]:
# # since our target has the highest correlation with the client category, we will take a closer look at that: 
# df_processed.client_catg.value_counts()

In [ ]:
# # plot distribution of target for each client category 

# fig = plt.figure(figsize=(20,10))
# sns.countplot(data= df_processed, x = 'client_catg', hue='target');

In [ ]:
# # plot distribution of target for each client category individually, descending by clients in category:

# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 11'), x = 'client_catg', hue='target', stat='percent');

In [ ]:
# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 51'), x = 'client_catg', hue='target', stat='percent')

In [ ]:
# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 12'), x = 'client_catg', hue='target', stat='percent')

In [ ]:
# the highest amount of fraud happens by clients assigned to category 51, 
# so our very fist guess for a hypothesis and baseline mode could be:
# client category is the best predictor for fraud
# however, due tue the unbalanced data, this would be a very poor model.

## Next Steps
* run baseline model and print the different scores for it
* Drop old and new index
* Train-Test-Split
* Define Target y, and Features X
* Feature Engineering
* Dummy Creating
* Rescaling based on train, apply to test
* dropping other unused columns
* export X-test, X_train, y_test and y_train to a new notebook for modeling (find library to help us with that)
* individual modeling

## Feature Engineering

In [ ]:
# # month and year from date columns
# df_processed['creation_month'] = df_processed['creation_date'].dt.month.astype(int)
# df_processed['creation_year'] = df_processed['creation_date'].dt.year.astype(int)
# df_processed['invoice_month'] = df_processed['invoice_date'].dt.month.astype(int)
# df_processed['invoice_year'] = df_processed['invoice_date'].dt.year.astype(int)

In [ ]:
# number of years as client

df_processed['member_years'] = df_processed['invoice_date'].dt.year.astype(int) - df_processed['creation_date'].dt.year.astype(int)
df_processed.head()



In [ ]:
# drop creation_date and invoice_date
df_processed.drop(['creation_date','invoice_date'] , inplace=True,axis=1)
df_processed.head()

In [ ]:
# tarif type valuecount

df_processed['tarif_type'].value_counts()



In [ ]:
# bin tariff


df_processed['tarif_type'] = df_processed['tarif_type'].map({
    
    11 : 11,
    40 : 40,
    10 : 10,
    15  : 15,
    45   : 45,
    13  : 13,
    14  : 14,
    12  : 12,
    29  : 29,
    9  : 0,
    21  :0,
    8   : 0,
    30  : 0,
    24  : 0,
    18   : 0,
    42  : 0,
    27  :0
    
    })





In [ ]:
# new feature index_change
df_processed['index_change']= df_processed['new_index'] - df_processed['old_index']

In [ ]:
df_processed.query('months_number == 0').T

In [ ]:
df_processed.query('client_id == "train_Client_86638"')

In [ ]:
# delete client with 0 consommation, 0 months_number...
df_processed.drop(index=3985967 , inplace=True,axis=0)

In [ ]:
df_processed.query('months_number == 0')

In [ ]:
# index change per month


df_processed['index_change_month']= df_processed['index_change'] / df_processed['months_number']




In [ ]:
# drop all  columns 
# Consummation_level_1 through 4, district, invoice_date, creation_date, counter_coefficient, index_change, months_number

df_processed.drop(['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4',
                'disrict',  'counter_coefficient', 'index_change', 'months_number', 'counter_code'], inplace=True, axis=1)

In [ ]:
df_processed.head()

In [ ]:
#fig = plt.figure(figsize=(20,10))
#sns.histplot(data=df_processed, x='index_change' ,hue='target')

In [ ]:
df_processed['index_change_month'].corr(df_processed['target']) #  correlation is very low 

In [ ]:
df_processed[['index_change_month']].describe()

In [ ]:
df_processed.drop(['old_index','new_index'] , inplace=True,axis=1)


In [ ]:
df_processed

In [ ]:
# counter per client feature
quantity_counter = df_processed.groupby('client_id')['counter_number'].count().reset_index()
quantity_counter.rename(columns={'counter_number':'quantity_counters'}, inplace=True)
quantity_counter.head()

In [ ]:
df_processed = pd.merge(df_processed, quantity_counter, on="client_id", how="left")
df_processed.head()

In [ ]:
df_processed['quantity_counters'].corr(df_processed['target']) #  correlation is very low 

In [ ]:
# fig = plt.figure(figsize=(20,10))
# sns.histplot(df_processed, x='quantity_counters', hue='target')

In [ ]:
# drop client_id and counter_number
df_processed.drop(['client_id','counter_number'] , inplace=True,axis=1)
df_processed.head()

In [ ]:
# rename district column
df_processed.rename(columns={'disrict':'district'}, inplace=True)

## Heatmap with the new features

In [ ]:
df_processed.columns

In [ ]:
# heatmap
fig = plt.figure(figsize=(20,10))
sns.heatmap(df_processed[['client_catg', 'region', 'target', 'tarif_type', 'counter_statue',
       'reading_remarque',  'member_years',
       'index_change_month', 'quantity_counters']].corr(), annot=True)


## Turning Categorical Features into Dummies

In [ ]:
df_processed.columns.to_list()

In [ ]:
cat_features = ['client_catg',
 'region',
 'tarif_type',
 'counter_statue',
 'reading_remarque',
 'counter_type']

In [ ]:
num_features = [x for x in df_processed.columns.to_list() if x not in cat_features]
num_features

In [ ]:
# remove target
num_features.remove("target")
num_features

In [ ]:
#make dummy variables from categorical features, dtype int
dummies = pd.get_dummies(df_processed[cat_features], columns=cat_features, drop_first=True, dtype=int)
dummies

In [ ]:
#%store -z

In [ ]:
# add dummy variables to dataframe with dropped categorical columns
df_processed_dmy =  pd.concat([df_processed.drop(cat_features,axis=1), dummies],axis=1)
df_processed_dmy.head()

In [ ]:
df_processed_dmy.shape

In [ ]:
# store the preprocessed dataframe
df_processed_dmy.to_csv('data/df_processed_dmy.csv', index=False)

# # add to new notebooks:
# # load the preprocessed dataframe
# %store -r df_processed_dmy

In [ ]:
#%store -z

## Turning Categorical Features into Dummys for Tree-Models (no drop-first)

In [ ]:
#make dummy variables from categorical features, dtype int, without dropping the first dummy columns
dummies_tree = pd.get_dummies(df_processed[cat_features], columns=cat_features, drop_first=False, dtype=int)
dummies_tree

In [ ]:
# add dummy variables to dataframe with dropped categorical columns
df_processed_dmy_tree =  pd.concat([df_processed.drop(cat_features,axis=1), dummies_tree],axis=1)
df_processed_dmy_tree.head()

In [ ]:
# store the preprocessed dataframe for tree models
df_processed_dmy_tree.to_csv('data/df_processed_dmy_tree.csv', index=False)

## To Dos

* counter_code: find info on steg site or drop?
* reading_remarque: clean? 
* counter_coefficient: try both?: treat numerically and as dummies?
* rescale consommation_level1 ... _4
* rescale months_number

## Done
* rename 'disrict' column 'district'
* turn creation_date, invoice_date  into datetime
* change target to 0,1
* counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove
* feature engineering: index_change = new_idex - old_index. then drop new_index, old_index
* feature engineering: number of counters per client. then drop client_id, counter_number
* turn into dummies: cat_features = ['district', 'client_catg', 'region', 'tarif_type', 'counter_statue', 'counter_code', 'reading_remarque', 'counter_coefficient', 'counter_type']
* export processed dataframe
* turn date columns into months and year columns


In [ ]:
# save current version of processed data for use later

#df_processed.to_csv('data/fraud_data_processed_V1.csv', index=False)


In [ ]:
#store num-features
%store num_features